#Actividad 3: Construcción de algoritmos de clasificación para análisis de sentimientos y reconocimiento de entidades.

**Introducción**

Esta actividad consiste en construir dos herramientas de procesamiento de lenguaje natural (PLN):

1.  **Analizador de Sentimiento:** Determina si un texto expresa un sentimiento positivo, negativo o neutral.
2.  **Etiquetador POS (Part-of-Speech):** Asigna a cada palabra de un texto su categoría gramatical (sustantivo, verbo, adjetivo, etc.).

Se utilizarán las siguientes librerías:

*   **TextBlob:** Para el análisis de sentimiento (una capa sobre NLTK que simplifica esta tarea).
*   **spaCy:** Para el POS tagging (una librería moderna y eficiente para PLN).
*   **NLTK:** Para el preprocesamiento de texto.

**Objetivos del Ejercicio**

1.  **Comprender Conceptos Clave:**
    *   Análisis de sentimiento (polaridad).
    *   POS tagging.
    *   Preprocesamiento de texto.
    *   Pruebas unitarias.

2.  **Desarrollar Habilidades de Programación:**
    *   Trabajar con librerías de PLN en Python (TextBlob, spaCy, NLTK).
    *   Implementar funciones de análisis de texto.
    *   Ejecutar pruebas unitarias.
    *   Trabajar colaborativamente.

3.  **Construir Herramientas Útiles:**
    *   Crear un sistema que pueda analizar el sentimiento de un texto.
    *   Crear un sistema que pueda etiquetar las palabras de un texto según su función gramatical.


## 1. Ejecute la siguiente celda para cargar las librerias nesesarias y descargar los complementos

In [1]:
import nltk
import spacy
from textblob import TextBlob
from collections import Counter
import re
import unittest

# Descarga de recursos de NLTK y Spacy
try:
    nltk.data.find('corpora/wordnet')
    nltk.data.find('corpora/omw-1.4')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('averaged_perceptron_tagger') # Para POS Tagging
except LookupError:
    nltk.download('wordnet')
    nltk.download('omw-1.4')
    nltk.download('stopwords')
    nltk.download('punkt')
    nltk.download('averaged_perceptron_tagger')

try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading en_core_web_sm model...")
    !python -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")


from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [1]:
!python -m spacy download en_core_web_sm  #En consola, fuera del notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 25.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


##2. Ejecute la siguiente definiendo los parametros restantes



*   **`umbral_polaridad_positiva`**: Este parámetro define el umbral de polaridad por encima del cual un texto se considerará "positivo". La polaridad es un valor numérico que va de -1 (muy negativo) a +1 (muy positivo), donde 0 indica neutralidad.
*   **`umbral_polaridad_negativa`**: Define el umbral de polaridad por debajo del cual un texto se considerará "negativo".
*   **`frecuencia_minima`**: Este parámetro se usa en la función pos_tagging. Determina la frecuencia mínima que debe tener una palabra en el texto para que se le asigne una etiqueta POS (Part-of-Speech). Si una palabra aparece menos veces que este valor, no se incluirá en el resultado del etiquetado POS (Este valor dejenlo en 1 para que evalue todas las palabras de la oración).



In [2]:
# --- Configuración (Parámetros) ---
#@markdown ### **Configuración de Parámetros**

#@markdown **Opciones de Análisis de Sentimiento:**
umbral_polaridad_positiva = 0.1  #@param {type:"number"}
umbral_polaridad_negativa = -0.1 #@param {type:"number"}

#@markdown **Opciones de Análisis de Texto (para POS Tagging):**
frecuencia_minima = 1  #@param {type:"integer"}


##3. Realice el codigo de las siguientes funciones y ejecute esa celda y la de las pruebas unitarias para comprobar su validez.

A continuación, se describen las funciones que debes implementar.  Para cada función, se proporciona:

*   **Nombre de la función y parámetros:**  Incluye los tipos de datos esperados.
*   **Docstring:** Una descripción detallada del propósito de la función, sus argumentos y el valor de retorno esperado.  *Lee esto con atención*.
*   **Esqueleto de la función:** El código básico de la función, con un `pass` donde debes escribir tu implementación.
* **Pruebas unitarias:** Recuerda que cada funcion cuenta con una prueba unitaria para probar que tu codigo funcione correctamente.

**Importante:**

*   **No modifiques los nombres de las funciones ni los parámetros.**  Las pruebas unitarias dependen de ellos.
*   **Lee cuidadosamente los docstrings.**  Contienen información esencial.
*   **Reemplaza el `pass` con tu código.**
*   **Ejecuta las pruebas unitarias después de implementar cada función.**

**Ejemplo:**

```
def Nombre de la funcion(parametros):
    """
    Docstring

    Args:
        Entradas.

    Returns:
        salidas (return ...) .
    """
    # --- IMPLEMENTAR AQUÍ ---
    pass

```

In [26]:
def analyze_sentiment(text):
    """
    Analiza el sentimiento de un texto usando TextBlob.

    Args:
        text (str): El texto a analizar.

    Returns:
        str:  "positive", "negative", o "neutral", según la polaridad del texto.
    """
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity

    if polarity > umbral_polaridad_positiva:
        return "positive"
    elif polarity < umbral_polaridad_negativa:
        return "negative"
    else:
        return "neutral"

def pos_tagging(text):
    """
    Realiza POS tagging usando Spacy, filtrando por frecuencia mínima.

    Args:
        text (str): El texto a analizar.

    Returns:
        dict: Un diccionario donde las claves son las palabras (que cumplen la
              frecuencia mínima) y los valores son sus POS tags.
              Si no hay palabras, devuelve un diccionario vacío.
    """
    doc = nlp(text)
    words = [token.text for token in doc]
    freq = Counter(words)

    pos_tags = {}
    for token in doc:
        if freq[token.text] >= frecuencia_minima:
            pos_tags[token.text.lower()] = token.pos_

    return pos_tags


In [27]:
#@markdown ### **Pruebas Unitarias**

# --- Pruebas Unitarias ---
class TestFunctions(unittest.TestCase):

    def test_analyze_sentiment(self):
        test_cases = [
            ("This is a great movie!", "positive"),
            ("I hate this product.", "negative"),
            ("The weather is okay.", "positive"),
            ("", "neutral"),  # Caso de texto vacío
            ("This is absolutely amazing!!!", "positive"),
            ("This is absolutely terrible!!!", "negative"),
            ("This movie is neither good nor bad.", "neutral")
        ]
        for text, expected in test_cases:
            with self.subTest(text=text):
                result = analyze_sentiment(text)
                self.assertEqual(result, expected)

    def test_pos_tagging(self):
        test_cases = [
            ("The quick brown fox jumps over the lazy dog.",
             {'the': 'DET','quick': 'ADJ','brown': 'ADJ', 'fox': 'NOUN', 'jumps': 'VERB', 'over': 'ADP', 'lazy': 'ADJ','dog': 'NOUN','.': 'PUNCT'}),  #Caso simple
            ("I love pizza.", {'i': 'PRON', 'love': 'VERB', 'pizza': 'NOUN', '.': 'PUNCT'}), #Ninguna palabra con frecuencia minima.
            ("apple apple apple", {"apple":"NOUN"}), #Caso con una sola palabra.
            ("", {}),  # Caso de texto vacío
            ("This is a sentence.  This is another sentence.",
             {'this': 'PRON','is': 'AUX', 'a': 'DET', 'sentence': 'NOUN', '.': 'PUNCT', ' ': 'SPACE', 'another': 'DET'}) #Caso con puntuacion.
        ]
        for text, expected in test_cases:
            with self.subTest(text=text):
                result = pos_tagging(text)
                self.assertEqual(result, expected)


# --- Ejecutar pruebas unitarias y luego la función principal si las pruebas pasan ---
if __name__ == '__main__':
    suite = unittest.TestLoader().loadTestsFromTestCase(TestFunctions)
    result = unittest.TextTestRunner().run(suite)
    if result.wasSuccessful():
        print("\n--- Todas las pruebas pasaron. Ejecutando la función principal... ---\n")
    else:
        print("\n--- Algunas pruebas fallaron. No se ejecutará la función principal. ---\n")

..
----------------------------------------------------------------------
Ran 2 tests in 0.038s

OK



--- Todas las pruebas pasaron. Ejecutando la función principal... ---



##6. Realice algunas pruebas del codigo final con diferentes textos.


In [28]:
#@title Prueba con texto

# --- Función Principal (para Colab) ---

def preprocess_text(text):
    """
    Preprocesa el texto: tokeniza, convierte a minúsculas, elimina
    signos de puntuación, caracteres no alfanuméricos y stop words.

    Args:
        text (str): El texto a preprocesar.

    Returns:
        list: Una lista de palabras preprocesadas.
    """
    stop_words = set(stopwords.words('english'))  # Usar 'english' o el idioma apropiado
    words = word_tokenize(text)
    words = [word.lower() for word in words]
    # Eliminar puntuación y caracteres no alfanuméricos con expresiones regulares
    words = [re.sub(r'[^a-zA-Z0-9\s]', '', word) for word in words]
    words = [word for word in words if word.isalnum()]  # Mantener solo alfanuméricos
    words = [word for word in words if word not in stop_words and word != '']
    return words

def main():
    #@markdown ### **Texto de Entrada**
    texto_entrada = "I really enjoy learning new things, but sometimes it can be frustrating when I don't understand a topic right away." #@param {type:"string"}

    if texto_entrada == "":
        texto_entrada = "This is a great and amazing movie! I really love it. But, this other movie is terrible and awful."

    print("--- Análisis de Sentimiento ---")
    sentimiento = analyze_sentiment(texto_entrada)
    print(f"Sentimiento general del texto: {sentimiento}")

    print(f"\n--- POS Tagging (palabras con frecuencia mínima de {frecuencia_minima}) ---")
    pos_tags = pos_tagging(texto_entrada)
    if pos_tags:
        for word, tag in pos_tags.items():
            print(f"  '{word}': {tag}")
    else:
        print("No se encontraron palabras que cumplan la frecuencia mínima.")

# --- Ejecutar la función principal ---
if __name__ == "__main__":
    main()

--- Análisis de Sentimiento ---
Sentimiento general del texto: positive

--- POS Tagging (palabras con frecuencia mínima de 1) ---
  'i': PRON
  'really': ADV
  'enjoy': VERB
  'learning': VERB
  'new': ADJ
  'things': NOUN
  ',': PUNCT
  'but': CCONJ
  'sometimes': ADV
  'it': PRON
  'can': AUX
  'be': AUX
  'frustrating': ADJ
  'when': SCONJ
  'do': AUX
  'n't': PART
  'understand': VERB
  'a': DET
  'topic': NOUN
  'right': ADV
  'away': ADV
  '.': PUNCT


In [18]:
#positivo
print(analyze_sentiment("I absolutely loved the new book. It was inspiring and beautifully written!"))

#negativo
print(analyze_sentiment("The service at the restaurant was terrible and the food was cold."))

#neutro
print(analyze_sentiment("The report was submitted on Monday and reviewed on Wednesday."))


positive
negative
neutral


In [22]:
text = "They can fish, but the can smells bad"
print(pos_tagging(text))

{'they': 'PRON', 'can': 'AUX', 'fish': 'VERB', ',': 'PUNCT', 'but': 'CCONJ', 'the': 'PRON', 'smells': 'VERB', 'bad': 'ADJ'}


In [24]:
text = "The record was broken, so they will record a new version tomorrow."
print(pos_tagging(text))

{'the': 'DET', 'record': 'VERB', 'was': 'AUX', 'broken': 'VERB', ',': 'PUNCT', 'so': 'SCONJ', 'they': 'PRON', 'will': 'AUX', 'a': 'DET', 'new': 'ADJ', 'version': 'NOUN', 'tomorrow': 'NOUN', '.': 'PUNCT'}


In [25]:
text = "Suddenly, the quick brown fox jumps over the lazy dog and barks loudly as it runs through the forest."
print(pos_tagging(text))

{'suddenly': 'ADV', ',': 'PUNCT', 'the': 'DET', 'quick': 'ADJ', 'brown': 'ADJ', 'fox': 'NOUN', 'jumps': 'VERB', 'over': 'ADP', 'lazy': 'ADJ', 'dog': 'NOUN', 'and': 'CCONJ', 'barks': 'VERB', 'loudly': 'ADV', 'as': 'SCONJ', 'it': 'PRON', 'runs': 'VERB', 'through': 'ADP', 'forest': 'NOUN', '.': 'PUNCT'}


**Conclusión:**

El análisis de sentimiento y el etiquetado gramatical son herramientas esenciales en el procesamiento del lenguaje natural, ya que a través del análisis de sentimiento, es posible clasificar opiniones o emociones expresadas en un texto, lo cual es útil en áreas como atención al cliente, redes sociales o análisis de productos y por otro lado, el POS tagging permite identificar la función gramatical de cada palabra, lo que facilita tareas como la traducción automática, el resumen de textos o la extracción de información. Esta actividad permitió evidenciar cómo una misma palabra puede tener distintos roles dependiendo del contexto, y cómo la diversidad de categorías gramaticales enriquece la interpretación del lenguaje.

Estas técnicas, implementadas correctamente, permiten a las máquinas comprender y manipular el lenguaje humano con mayor precisión y contexto.